# Inheritance and Subtype Polymorphism
## Inheritance Overview
###### Let's take a deeper look at Python's support for inheritance including multiple inheritance and the underlying mechanisms by which Python dispatches method calls. We'll also look at techniques for runtime-type checking for those times when it's necessary. Before we look at multiple inheritance, let's do a quick review of single inheritance. The syntax of single inheritance is part of the class declaration with the BaseClass put in parentheses after the class name. What this essentially means is that SubClass will have all of the methods of BaseClass, and SubClass will be able to override these methods if it wants to. In other words, SubClass can do everything that BaseClass can do, and it can optionally modify or specialize that behavior. In general, a subclass initializer will want to call its base class initializer to make sure that the full object is initialized. Remember though that if a subclass doesn't define an initializer then the base class initializer is called when an instance of the subclass is created. To see the basics of inheritance in action, let's define a very simple base class called Base.  If we create an instance of Base, we see of course that its initializer is called, and calling the f() method on that instance uses Base.f(). There should be nothing surprising here. Let's now define a subclass of Base called Sub. This subclass doesn't add any functionality to Base at all. Because we haven't defined an initializer for Sub, we can see that it inherits Base's initializer. And likewise, Sub also inherits Base.f(). Sub can override Base.f() by simply redefining the method itself. Now, if we create an instance of Sub, we see that the base initializer is still being called, but Sub's definition of F is now used. Finally, let's give Sub its own initializer. Now when we create a new instance of Sub we only see Sub's initializer being called. If you're used to languages like C++ and Java, you might have expected Python to also call Base's initializer when creating a Sub instance, but this isn't how Python behaves. Rather, Python treats the dunder-init method just like any other method, and it doesn't automatically call base class initializers for subclasses that define their own initializers. If you define an initializer in a subclass and still want to call the initializer of a base class, and this is often important to do, then you need to call it explicitly using the super() function. Let's finish this small example by doing that. Now when we construct an instance of Sub we see both the Sub and Base initializers being called. The fact that Python doesn't provide special support for calling initializers is important to understand, and if it seems strange to you don't worry. It soon becomes second nature. Hopefully this small example was just review for you.

In [2]:
class Base:
    def __init__(self):
        print('Base initializer')
        
    def f(self):
        print('f.base')
        

In [3]:
b =Base()

Base initializer


In [4]:
b.f()

f.base


In [5]:
class Base:
    def __init__(self):
        print('Base initializer')
        
    def f(self):
        print('f.base')
        
# No functionality to base at all        
class Sub(Base):
    pass

In [7]:
# Notice it calls the base initializer in absence of sub class initializer
s = Sub()

Base initializer


In [9]:
# Also inherits base f function
s.f()

f.base


In [13]:
# Sub can overide by redifing sub
class Base:
    def __init__(self):
        print('Base initializer')
        
    def f(self):
        print('f.base')
        
# No functionality to base at all        
class Sub(Base):
    
    def f(self):
        print('f.sub')

In [14]:
s= Sub()

Base initializer


In [15]:
s.f()

f.sub


In [16]:
# lets give sub its own intializer
# Sub can overide by redifing sub
class Base:
    def __init__(self):
        print('Base initializer')
        
    def f(self):
        print('f.base')
        
# No functionality to base at all        
class Sub(Base):
    
    def __init__(self):
        print('Sub intialiser')
    
    def f(self):
        print('f.sub')

In [17]:
s= Sub()

Sub intialiser


In [18]:
s.f()

f.sub


In [29]:
# lets use the super
# so that it calls the base init
# in C++ it is done automatically
# lets give sub its own intializer
# Sub can overide by redifing sub
class Base:
    def __init__(self):
        print('Base initializer')
        
    def f(self):
        print('f.base')
        
# No functionality to base at all        
class Sub(Base):
    
    def __init__(self):
        super().__init__()
        print('Sub intialiser')
    
    def f(self):
        print('f.sub')


In [31]:
# Both initialzer is called 
s= Sub()

Base initializer
Sub intialiser


# A Realistic Example: SortedList
###### As a more concrete and practical example of how to use inheritance in Python, we'll first define our own simplified list class called SimpleList. SimpleList uses a standard list internally, and it provides a smaller, more limited API for interacting with the list's data. We'll use SimpleList as the basis for the rest of our exploration of inheritance in Python. Next let's create a subclass of SimpleList which keeps the list contents sorted. We'll call this class SortedList, and it looks like this. You can see that the class declaration includes the class name followed by SimpleList in parentheses, so SimpleList is the base class of SortedList. The initializer for SortedList takes an optional argument, which is a sequence for initializing the list's contents. The initializer calls SimpleList's initializer and then immediately uses SimpleList.sort to sort the contents. SortedList also overrides the add method on SimpleList to ensure that the list always remains sorted. One aspect of this example that you'll notice are the calls to super(). We'll be covering super() in more detail later, but in this case it can be understood to mean call a method on my base class. So, for example, calling super.add X in SortedList.add means to simply call SimpleList.add X with the same self argument or in other words to use the base-class implementation of add. If we go to our REPL, we can see that SortedList works as expected. (Typing)

In [32]:
class SimpleList:
    def __init__(self,items):
        self._items = list(items)
        
    def add(self,item):
        self._items.append(item)
        
    def __getitem__(self,index):
        return self._items[index]
    
    def sort(self):
        self._items.sort()
        
    def __len__(self):
        return len(self._items)
    
    def __repr__(self):
        return "SimpleList({!r})".format(self._items)

In [33]:
class SortedList(SimpleList):
    def __init__(self,items=()):
        super().__init__(items)
        self.sort()
        
    def add(self,item):
        super().add(item)
        self.sort()
    
    def __repr__(self):
        return "SortedList({!r})".format(self._items)
    

In [34]:
sl = SortedList([3,4,11,78])

In [35]:
len(sl)

4

In [36]:
sl.add(-42)

In [37]:
sl

SortedList([-42, 3, 4, 11, 78])

# The Built-In isinstance() Function
###### Single inheritance in Python is relatively simple and should be conceptually familiar to anyone who's worked with almost any other object-oriented language. Multiple inheritance in Python is not much more complex, and as we'll eventually see both single and multiple inheritance ultimately rely on a single underlying model. Before we look at multiple inheritance though, we need to lay some ground work for the examples we'll be using. Along with the SortedList class we defined earlier, we're going to define another class, IntList, which only allows integer contents. This list subclass will prevent the insertion of non-integer elements. To do this, IntList will need to check the types of items that are inserted, and the tool it will use for this is the isinstance() built-in function. Isinstance() takes an object as its first argument and a type as its second. It then determines if the object is of the specified type returning true if it is and false otherwise. For example, here we see isinstance() applied to a number of built-in types. Instead of just checking for an exact typed match, isinstance() will also return true if the object is a subclass of the second argument. For example, we can see that a sorted list is an instance of SortedList, as well as of SimpleList. A final twist to isinstance() is that it can accept a tuple of types for its second argument. This is equivalent to asking if the first argument is an instance of any of the types in the tuple. For example, this call returns true because X is an instance of list. Now that we know how to use isinstance(), we can define our IntList class like this. You'll immediately notice that IntList is structurally similar to SortedList. It provides its own initializer and like SortedList overrides the add method to perform extra checks. In this case, IntList calls its _validate method on every item that goes into the list. _validate uses isinstance() to check the type of the candidates, and if a candidate is not an instance of int, _validate raises a TypeError. Let's see how this looks in the REPL. (Typing) So, we can see how isinstance() can be used to do type checks in Python. Checks like that are uncommon in Python, and while some people consider them to be a sign of poor design, sometimes they're simply the easiest way to solve a problem.

In [38]:
isinstance(3,int)

True

In [39]:
isinstance("hello",str)

True

In [40]:
isinstance(4.23,int)

False

In [41]:
x = []
# Any of the type
isinstance(x,(int,float,list))

True

In [42]:
class InList(SimpleList):
    def __init__(self,items=()):
        for x in items:
            self._validate(x)
        super().__init__(items)
        
    @staticmethod
    def _validate(x):
        if not isinstance(x,int):
            raise TypeError('Int list only supports int values')
            
    def add(self,item):
        self._validate(item)
        super().add(item)
        
    def __repr__(self):
        return "SortedList({!r})".format(self._items)
        

In [43]:
il = InList([1,2,3,4,5])

In [44]:
il.add(19)

In [45]:
il.add('5')

TypeError: Int list only supports int values

# The Built-In issubclass() Function
###### There is another built-in function related to isinstance(), which can be used for type checking. This function, issubclass(), operates on types only rather than operating on instances of types. As its name implies, issubclass() is used to determine if one class is a subclass of another. Issubclass() takes two arguments, both of which need to be type objects, and it returns true if the first argument is a direct or indirect subclass of the second. For example, we can see that both IntList and SortedList are subclasses of SimpleList as you would expect. On the other hand, SortedList is not a subclass of IntList. We can also use a simple example to verify that issubclass() looks at the entire inheritance graph, not just direct parents. These classes are obviously pretty silly, but they do illustrate that issubclass() recognizes that MyVerySpecialInt is indeed a subclass of int even though it only directly inherits from MyInt.



In [46]:
issubclass(SortedList,SimpleList)

True

In [47]:
issubclass(InList,SimpleList)

True

In [48]:
issubclass(InList,SortedList)

False

# Multiple Inheritance
###### Now that we have defined two interesting subclasses of SimpleList, we're ready to take a look at multiple inheritance in Python. Multiple inheritance simply means defining classes with more than one direct base class. This feature is not universal among object-oriented languages. C++ supports multiple inheritance, for example, while Java does not. Multiple inheritance can lead to certain complex situations, for example, deciding what to do when more than one base class defines a particular method, but as we'll see Python has a relatively simple and understandable system for handling such cases. The syntax for defining a class with multiple inheritance is very similar to single inheritance. To declare multiple base classes, simply use a comma-separated list of classes in the parentheses after the class name. A class can have as many base classes as you want. Just like single inheritance, a class with multiple base classes supports all of the methods of its bases. As long as there is no overlap in the method names of the base classes, it's always easy to see how method calls are resolved. Find the base class with the matching method name, and that's which method gets called. If there is method name duplication across base classes, Python has a well-defined Method Resolution Order for determining which is called. We'll look at Method Resolution Order in more detail shortly. Let's jump right in and define our own class using multiple inheritance. So far in this module we've defined SortedList and IntList, both of which inherit from our SimpleList. Now we're going to define a class which inherits from both of these classes and thus has the properties of both. Here's our SortedIntList. It doesn't look like much does it? We've simply defined a new class and given it two base classes. In fact, the only new implementation in the class is there to give it a proper string representation. But if we go to the REPL, we can see that it works as we expect. The initializer sorts the input sequence, but rejects non-integer values. Likewise, the add method maintains both the sorting and type constraints defined by the base classes. You should spend some time playing with SortedIntList to convince yourself that it works as expected. It may not be immediately apparent how all of this works though. After all, both IntList and SortedList define add. How does Python know which add() to call? Or more importantly, since both the sorting and type constraints are being enforced by SortedIntList, how does Python seem to know to call both of them? The answers to these questions have to do with the method resolution order we mentioned earlier along with the details of how super() really works. We'll get to all of that very soon.

In [55]:
#Multiple inheritance
class SortedIntList(InList,SortedList):
    
    def __repr__(self):
        return "SortedIntList({!r})".format(list(self))

In [56]:
sil = SortedIntList([42,23,2])

In [57]:
sil

SortedIntList([2, 23, 42])

In [58]:
sil = SortedIntList([42,23,'2'])

TypeError: Int list only supports int values

In [59]:
sil.add(3)

In [60]:
sil

SortedIntList([2, 3, 23, 42])

In [61]:
# Which add is getting called ?

# Details of Multiple Inheritance
##### Before that, there are a few more details about multiple inheritance that we need to cover. First, if a class uses multiple inheritance, but defines no initializer, only the initializer of the first base class is automatically called when an instance of that class is created. Consider this simple example. (Typing) If we now create an instance of Sub(), we see that only the initializer for Base1 is called. Through the use of super() we could design these classes such that both the Base1 and Base2 initializers are called automatically, and we'll see how to do that soon. Another useful thing to know when thinking about inheritance is the dunder-bases member of class objects. Dunder-bases is simply a tuple of a class's base classes. As you can see here SortedIntList inherits from both IntList and SortedList, and these show up in the dunder- bases member of the SortedIntList class object. The entries in dunder-bases are in the same order as they were declared in the class definition. You don't have to use multiple inheritance to populate dunder- bases as you can see if you look at dunder-bases for our IntList class.

In [64]:
class Base1:
    def __init__(self):
        print("base1.init")
        
class Base2:
    def __init__(self):
        print("base2.init")
        
class Sub(Base1,Base2):
    pass
    

In [66]:
s =Sub()

base1.init


In [68]:
SortedIntList.__bases__

(__main__.InList, __main__.SortedList)

# Method Resolution Order
##### We're finally ready to talk about this notion of method resolution order that we've mentioned several times now. In Python the method resolution order or simply MRO of a class is the ordering of a class's inheritance graph used to determine which implementation to use when a method is invoked. When you invoke a method on an object which has one or more base classes, the actual code that gets run may be defined on the class itself, one of its base classes, a base class of a base class, or any other member of the class's inheritance graph. The MRO of a class determines the order in which the inheritance graph is searched to find the correct implementation of the method. That's a lot to take in, but MRO is actually very simple, and we'll look at some examples that will make things more clearer. First though, we need to look at where our class's MRO is stored. The method resolution order for a class is stored on a special member called dunder-mro. Here you can see the MRO for SortedIntList. The dunder-mro attribute is a tuple of classes defining the method resolution order. You can also call the method MRO on a class to get the same information in a list rather than as a tuple. So, how is the MRO used to dispatch method calls? The answer is that when you call a method on an object in Python, Python looks at the MRO for that object's type. For each entry in the MRO starting at the front and working in order to the back, Python checks if that class has the requested method. As soon as Python finds a class with a matching method, it uses that method and the search stops. Let's see a simple example. First we'll define a few classes with a diamond inheritance graph. Here the various func methods simply report which class they come from. If we look at D's MRO, we see that Python will check D first, then B, then C, followed by A, and finally object when resolving calls to objects of type D. By the way, object is the ultimate base class of every class in Python, and we'll discuss that more later in this module. Based on that MRO, what should we expect if we call func on an instance of D? Because B was the first class in D's MRO with the method func, Python called B.func. If C had been earlier in the MRO than B, then C.func would have been called. We can see this by changing the order of B and C in the definition of D. After this change the new MRO for D puts C before B, and indeed calling func on a new instance of D results in a call to C's implementation. That's all there really is to it. MRO is an ordering of a class's inheritance graph that Python calculates for you. When Python resolves a method call, it simply walks along that ordering until a class supports the requested method. Let's see the MRO for our SortedIntList class. As you might expect, the MRO is SortedIntList followed by IntList followed by SortedList with SimpleList and object bringing up the rear, so calls to add on a SortedIntList, for example, were resolved to IntList.add since IntList is the first class in the MRO with an add method. This raises a very interesting question, however. When we wrote IntList, it never had any connection to the SortedList class, yet our SortedIntList as we've seen is properly maintaining both the sorting constraint and the type constraint of both SortedList and IntList. If add resolves to IntList.add() and if IntList is using super() to call its base class implementation, how is SortedList being invoked to maintain the sorting? The answer to that mystery has to do with how super() actually works.

In [69]:
SortedIntList.__mro__

(__main__.SortedIntList,
 __main__.InList,
 __main__.SortedList,
 __main__.SimpleList,
 object)

In [70]:
SortedIntList.mro()

[__main__.SortedIntList,
 __main__.InList,
 __main__.SortedList,
 __main__.SimpleList,
 object]

In [74]:
class A:
    def func(self):
        return 'A.func'
    
class B(A):
    def func(self):
        return 'B.func'    
    
class C(A):
    def func(self):
        return 'C.func'    
    
class D(B,C):
    pass    
           

In [75]:
D.mro()

[__main__.D, __main__.B, __main__.C, __main__.A, object]

In [76]:
#What should we expect when we call  func on D
d =D()

In [77]:
d.func()

'B.func'

In [86]:
class A:
    def func(self):
        return 'A.func'
    
class B(A):
    def func(self):
        return 'B.func'    
    
class C(A):
    def func(self):
        return 'C.func'    
    
class D(C,B):
    pass    

In [87]:
D.mro()

[__main__.D, __main__.C, __main__.B, __main__.A, object]

In [88]:
d = D()
d.func()

'C.func'

In [89]:
SortedIntList.mro()

[__main__.SortedIntList,
 __main__.InList,
 __main__.SortedList,
 __main__.SimpleList,
 object]

# How is Method Resolution Order Calculated?
###### Before we move on to looking at super() in detail, you might be curious to know how Python actually calculates the MRO for a class. The short answer is that Python uses an algorithm known as C3 for determining MRO. We won't go into the details of C3 except to mention a few important qualities of the MRO that it produces. First, a C3 MRO ensures that the subclasses come before their base classes. Second, C3 ensures that the base class order as defined in class definition is also preserved. Finally, C3 preserves the first two qualities independent of where in an inheritance graph you calculate the MRO. In other words, the MROs for all classes in graph agree with respect to relative class order. One outcome of the C3 algorithm is that not all inheritance declarations are allowed in Python. That is, some base class declarations will violate C3, and Python will refuse to compile them. Consider this simple example in the REPL. Here since B and C both inherit from A, B and C must both come before A in any MRO. This follows from one of the qualities that C3 preserves. However, since D's base class declaration puts A before C and since C3 also guarantees that base class declaration order is preserved, C3 cannot produce a consistent MRO. That is, it can't put A both before and after C. Understanding C3 is not critical or really even necessary for using Python, but it is an interesting topic for those curious about language design. If you want to learn more about it, you can find plenty of information on the web.

In [91]:
# Consider this
class A:
    pass

class B(A):
    pass

class C(A):
    pass

# since B and C both inherit from A 
# B and C must come before A
class D(B,A,C):
    pass

TypeError: Cannot create a consistent method resolution
order (MRO) for bases A, C

In [92]:
# Consider this
class A:
    pass

class B(A):
    pass

class C(A):
    pass

# since B and C both inherit from A 
# B and C must come before A
class D(B,C,A):
    pass

# The Built-In super() Function
###### Finally we have enough background to understand super(). So far we've seen super() used to access methods on base classes, for example, in SortedList.add where we used super() to call SimpleList.add before sorting the contents. From this example you might conclude that super() somehow returns the base class of a method's class and that you can then invoke methods on the base class part of an object. This is only partly true. It's hard to sum up what super() does in a single sentence, but here's an attempt. Given a method resolution order and a class C in that MRO, super() gives you an object which resolves methods using only the part of the MRO which comes after C. In other words, super() doesn't work with the base classes of a class, but instead it works with the MRO of the type of the object on which a method was originally invoked. The distinction is subtle, but very important, and as we'll see it resolves the mystery of how SortedIntList works. First, let's look at the details of calling super(). Super() can be called in several ways, but all of them return a so-called super() proxy object. You can call any method on a super() proxy, and it will route the call to the correct method implementation for you if such a method exists. There are two high-level types of super() proxies, bound and unbound. Bound proxies, as the name suggests, are bound to instances or class objects. Unbound proxies are not bound, and in fact don't do any method dispatch themselves. Unbound proxies are primarily an implementation detail for other Python features. Some prominent Python developers consider them a bit of a wart on the language, and really they are beyond the scope of this course, so we won't discuss them in any more detail. Bound proxies, on the other hand, are an important part of mastering Python, so we'll discuss them in detail. From now on when I talk about proxies or super() proxies, understand that I'm talking about bound proxies.

# Class-Bound Super Proxies
###### So, how do we use super() to create proxies? As I mentioned a few seconds ago, bound proxies can be bound to either classes or instances of classes. I will call these class-bound and instance-bound proxies respectively. To create a class-bound proxy you use this form. Here both arguments are class objects. The second class must be a subclass of or the same class as the first argument. When you invoke a method on the proxy, here's what happens. First, Python finds the MRO for derived-class. Second, it then finds base-class in that MRO. Third, it takes everything after base-class in the MRO and finds the first class in that sequence with a method name matching the request. Let's see that in action with our SortedIntList. First, let's see the MRO for SortedIntList. It's SortedIntList, IntList, SortedList, SimpleList, and finally object. So, what do we get if we call super() with the arguments SortedList and SortedIntList? That gives us a proxy bound to the arguments we'd expect. Well, what if you use that proxy to resolve a method, say add? Applying the algorithm described above, Python first finds the MRO for the second argument, which we just printed a few seconds ago. It then finds SortedList in that MRO and takes everything after SortedList giving us an MRO containing just SimpleList and object. It then finds the first class in that MRO with an add method, which of course is SimpleList. Let's see if we're right. Well, there you have it. Super() returned a proxy, which when asked for an add method, returned add from the SimpleList class. Now that we have a method, we should be able to call it right? Aah, right. Our proxy is bound to a class, not an instance, so we can't invoke it. If we used the proxy to look up a static method or class method, however, we could invoke it directly. To illustrate this, let's use a class-bound super proxy to call the _validate static method on IntList. This is probably not the kind of code you'd write in practice, but it does show how super() works when bound to class objects. Note that Python will raise an exception if the second argument is not a subclass of the first.

In [93]:
SortedIntList.mro()

[__main__.SortedIntList,
 __main__.InList,
 __main__.SortedList,
 __main__.SimpleList,
 object]

In [94]:
super(SortedList,SortedIntList)

<super: __main__.SortedList, __main__.SortedIntList>

In [96]:
# lets find out 
super(SortedList,SortedIntList).add

<function __main__.SimpleList.add>

In [99]:
super(SortedIntList,SortedIntList)._validate(5)

In [100]:
super(SortedIntList,SortedIntList)._validate("hello")

TypeError: Int list only supports int values

# Instance-Bound Super Proxies
##### Instance-bound super proxies work very much like class-bound proxies, but instead of binding to a class object they bind to an instance. To create an instance-bound proxy, call super() like this. Here the first argument must be a class object, and the second argument must be an instance of that class or any class derived from it. The behavior of the super proxy in this case is like this. First, Python finds the MRO for the type of the second argument. Second, Python finds the location of the first argument to super in that MRO. Remember that the instance must be derived from the class, so the class must be in the MRO. Third, Python finally takes everything in the MRO after the class and uses that as the MRO for resolving methods. Importantly, since the proxy is bound to an instance, we can call the methods after they've been found. Let's try that with our SortedIntList example. (Typing) So, the proxy is bound to a SortedIntList and will start method resolution from SortedIntList's MRO at the point after SortedList. As you'll recall, the next entry in SortedIntList's MRO after SortedList is SimpleList, so this super proxy will be directly using SimpleList's methods bypassing our constraint checks. Let's see if that works. Oh dear. Our SortedIntList not only isn't sorted anymore, but it also contains references to classic British TV. Clearly you need to use super() with care or you can end up breaking your designs.

In [101]:
sil = SortedIntList([5,15,10])

In [102]:
sil

SortedIntList([5, 10, 15])

In [104]:
SortedIntList.mro()

[__main__.SortedIntList,
 __main__.InList,
 __main__.SortedList,
 __main__.SimpleList,
 object]

In [103]:
super(SortedList,sil)

<super: __main__.SortedList, SortedIntList([5, 10, 15])>

In [105]:
super(SortedList,sil).add(6)

In [106]:
sil


SortedIntList([5, 10, 15, 6])

In [107]:
#Be sure to have a proper user of super otherwise it may 
# Break your design 

# Calling super() Without Arguments
###### We now know how super() works for creating class and instance-bound proxy objects, and we've seen both in action. The examples we've shown though all pass parameters to super() while the examples we saw earlier in this module, for example in the implementation of SortedIntList, didn't use any parameters at all. It turns our that you can call super() in a method with no arguments, and Python will sort out the arguments for you. If you're in an instance method, that is a method which takes an instance as its first argument and you call super() without arguments, that's the same as calling super() with the method's class as the first argument and self as the second. In the simple case of single inheritance then, this is equivalent to looking for a method on the base class. If you call super() without arguments in a class method, Python sets the arguments for you so that it's equivalent to calling super() with the method's class as the first argument and the method's first argument, that is the class argument, as the second. Again, in the typical case this is equivalent to calling the base class's method. In both cases then, for instance methods and class methods, calling super with no arguments puts the method's class as the first argument to super() and the first argument to the method itself as the second argument to super.

In [108]:
class nothing:
    pass

In [109]:
nothing.__base__

object

In [110]:
dir(object)

['__class__',
 '__delattr__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__']